<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
ANN Architecture Performance Analysis
</p>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# Load the DataFrame
model_results_df = pd.read_csv("model_results_summary.csv")

# Define the function to sum hidden layers
def sum_hidden_layers(hidden_layers_str):
    # Convert string representation of list to an actual list
    hidden_layers = ast.literal_eval(hidden_layers_str)
    # Calculate and return the sum
    return sum(hidden_layers)
    
# Define a function to count the number of hidden layers
def count_hidden_layers(hidden_layers_str):
    # Convert string representation of list to an actual list
    hidden_layers = ast.literal_eval(hidden_layers_str)
    # Return the number of layers
    return len(hidden_layers)

# Apply the function to create 'Total Nodes' column
model_results_df['Total Nodes'] = model_results_df['Hidden Layers'].apply(sum_hidden_layers)

# Apply the function to create 'Number of Layers' column
model_results_df['Number of Layers'] = model_results_df['Hidden Layers'].apply(count_hidden_layers)

# Update the desired column order
desired_order = [
    'Architecture',
    'Hidden Layers',
    'Number of Layers',
    'Total Nodes',
    'Test RMSE',
    'R2 Score',
    'Train MSE',
    'Train MAE',
    'Test MSE',
    'Test MAE',
    'Test MAPE (%)',
    'Test Median AE'
]

# Reorder the DataFrame columns
model_results_df = model_results_df[desired_order]

model_results_df.head()

,Architecture,Hidden Layers,Number of Layers,Total Nodes,Test RMSE,R2 Score,Train MSE,Train MAE,Test MSE,Test MAE,Test MAPE (%),Test Median AE
0,A1,[1],1,1,9.685326,0.736093,92.811363,7.110074,93.810211,7.134353,8.182885,5.711137
1,A2,[2],1,2,7.879093,0.825348,61.358120,5.899765,62.084599,5.918158,6.779106,4.947034
2,A3,[3],1,3,8.472686,0.798041,70.846672,6.324357,71.791901,6.344282,7.420802,5.182812
3,A4,[4],1,4,6.988565,0.862596,48.112999,5.131922,48.858871,5.154155,5.898322,4.167120
4,A5,[5],1,5,6.762546,0.871340,45.072857,4.915048,45.743378,4.937366,5.672196,3.935739


In [8]:
# Sort the DataFrame by 'Test RMSE' in ascending order
model_results_df_sorted = model_results_df.sort_values(by='Test RMSE', ascending=True)

# Prepare customdata with 'R2 Score', 'Number of Layers', and 'Hidden Layers'
customdata = model_results_df_sorted[['R2 Score', 'Number of Layers', 'Hidden Layers']].values

# Map 'Number of Layers' to colors
color_map = {1: 'skyblue', 2: 'lightgreen'}
colors = model_results_df_sorted['Number of Layers'].map(color_map)

# Create a Plotly figure with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Trace for Test RMSE (interactive), colored by 'Number of Layers'
fig.add_trace(
    go.Bar(
        x=model_results_df_sorted['Architecture'],
        y=model_results_df_sorted['Test RMSE'],
        name='Test RMSE',
        marker_color=colors,
        text=model_results_df_sorted['Total Nodes'],
        hoverinfo='none',  # Suppress default hover labels
        hovertemplate=(
            '<b>Architecture:</b> %{customdata[2]}<br>'
            '<b>Test RMSE:</b> %{y:.2f}<br>'
            '<b>R² Score:</b> %{customdata[0]:.4f}<br>'
            '<b>Total Nodes:</b> %{text}<br>'
            '<b>Number of Layers:</b> %{customdata[1]}<br>'
            '<extra></extra>'  # Suppress trace name in hover
        ),
        customdata=customdata,
    ),
    secondary_y=False,
)

# Dummy scatter traces for the legend
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color='skyblue'),
        legendgroup='1 Hidden Layer',
        showlegend=True,
        name='1 Hidden Layer'
    )
)

fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color='lightgreen'),
        legendgroup='2 Hidden Layers',
        showlegend=True,
        name='2 Hidden Layers'
    )
)

# Trace for R² Score (non-interactive)
fig.add_trace(
    go.Scatter(
        x=model_results_df_sorted['Architecture'],
        y=model_results_df_sorted['R2 Score'],
        name='R² Score',
        mode='lines+markers',
        line=dict(color='red', width=2),
        marker=dict(size=8),
        hoverinfo='skip',  # Disable hover interaction for R² Score
    ),
    secondary_y=True,  # Move 'secondary_y=True' here
)

# x-axis labels to display all architectures
fig.update_xaxes(
    title_text='Architecture',
    tickmode='array',
    tickvals=model_results_df_sorted['Architecture'],
    ticktext=model_results_df_sorted['Architecture'],
    tickangle=-45,    # Rotate labels for readability
)

# y-axes titles and colors
fig.update_yaxes(title_text='Test RMSE', secondary_y=False, color='blue')
fig.update_yaxes(title_text='R² Score', secondary_y=True, color='red')

# The layout with legend and centered title
fig.update_layout(
    title=dict(
        text='Interactive Test RMSE and R² Score for Neural Network Architectures',
        x=0.5,  # Center the title
        xanchor='center',  # Anchor the title to the center
    ),
    barmode='group',  # Use group mode for clarity
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.1,
        xanchor='center',
        x=0.5,
        bgcolor='rgba(255,255,255,0.8)',  # Semi-transparent background
    ),
    hovermode='x unified',
)

# Display the figure
fig.show()

In [9]:
# Get the 'Test RMSE' value for architecture 'C4'
c4_test_rmse = model_results_df.loc[model_results_df['Architecture'] == 'C4', 'Test RMSE'].values[0]

# Filter the DataFrame
model_results_df_filtered = model_results_df[
    (model_results_df['Test RMSE'] <= c4_test_rmse) &  # C4 condition
    (
        (model_results_df['Number of Layers'] != 1) |  # Exclude Total Nodes >= 6 for Number of Layers = 1
        (model_results_df['Total Nodes'] < 6)
    ) & 
    (model_results_df['Total Nodes'] < 9)  # General condition for Total Nodes < 9
].copy()

# Display the filtered DataFrame
print("Filtered DataFrame (with C4 condition and layer-specific Total Nodes filtering):")
model_results_df_filtered.head()

Filtered DataFrame (with C4 condition and layer-specific Total Nodes filtering):


,Architecture,Hidden Layers,Number of Layers,Total Nodes,Test RMSE,R2 Score,Train MSE,Train MAE,Test MSE,Test MAE,Test MAPE (%),Test Median AE
1,A2,[2],1,2,7.879093,0.825348,61.358120,5.899765,62.084599,5.918158,6.779106,4.947034
3,A4,[4],1,4,6.988565,0.862596,48.112999,5.131922,48.858871,5.154155,5.898322,4.167120
4,A5,[5],1,5,6.762546,0.871340,45.072857,4.915048,45.743378,4.937366,5.672196,3.935739
20,C4,"[2, 4]",2,6,7.908241,0.824053,62.083420,6.151520,62.551243,6.168030,7.264230,5.302179
26,D3,"[3, 3]",2,6,7.715153,0.832540,58.911507,5.924065,59.537094,5.942459,6.827964,5.060822


In [10]:
# Sort the filtered DataFrame by 'Test RMSE' in ascending order
model_results_df_filtered_sorted = model_results_df_filtered.sort_values(by='Test RMSE', ascending=True)

# Prepare customdata with 'R2 Score', 'Number of Layers', and 'Hidden Layers'
customdata_filtered = model_results_df_filtered_sorted[['R2 Score', 'Number of Layers', 'Hidden Layers']].values

# Map 'Number of Layers' to colors
color_map_filtered = {1: 'skyblue', 2: 'lightgreen'}
colors_filtered = model_results_df_filtered_sorted['Number of Layers'].map(color_map_filtered)

# Create a Plotly figure with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar trace for Test RMSE (interactive), colored by 'Number of Layers'
fig.add_trace(
    go.Bar(
        x=model_results_df_filtered_sorted['Hidden Layers'],  # x-axis remains Architecture for plotting
        y=model_results_df_filtered_sorted['Test RMSE'],
        name='Test RMSE',
        marker_color=colors_filtered,  # color mapping for filtered data
        marker_line_color='blue',
        marker_line_width=1.5,  # thickness
        #text=model_results_df_filtered_sorted['Total Nodes'],
        hoverinfo='none',  # Suppress default hover labels
        hovertemplate=(
            '<b>Architecture:</b> %{customdata[2]}<br>'  # Display Hidden Layers as Architecture
            '<b>Test RMSE:</b> %{y:.2f}<br>'
            '<b>R² Score:</b> %{customdata[0]:.4f}<br>'
            '<b>Total Nodes:</b> %{text}<br>'
            '<b>Number of Layers:</b> %{customdata[1]}<br>'
            '<extra></extra>'  # Suppress trace name in hover
        ),
        customdata=customdata_filtered,
    ),
    secondary_y=False,
)

# Add dummy scatter traces for the legend to represent different 'Number of Layers'
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color='skyblue'),
        legendgroup='1 Hidden Layer',
        showlegend=True,
        name='1 Hidden Layer'
    )
)

fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color='lightgreen'),
        legendgroup='2 Hidden Layers',
        showlegend=True,
        name='2 Hidden Layers'
    )
)

# Add line trace for R² Score (non-interactive)
fig.add_trace(
    go.Scatter(
        x=model_results_df_filtered_sorted['Hidden Layers'],
        y=model_results_df_filtered_sorted['R2 Score'],
        name='R² Score',
        mode='lines+markers',
        line=dict(color='red', width=2),
        marker=dict(size=8),
        hoverinfo='skip',  # Disable hover interaction for R² Score
    ),
    secondary_y=True,
)

# Update x-axis labels to display all architectures
fig.update_xaxes(
    title_text='Architecture',
    tickmode='array',
    tickvals=model_results_df_filtered_sorted['Hidden Layers'],
    ticktext=model_results_df_filtered_sorted['Hidden Layers'],
    tickangle=-45,    # Rotate labels for readability
)

# Update y-axes titles and colors
fig.update_yaxes(title_text='Test RMSE', secondary_y=False, color='blue')
fig.update_yaxes(title_text='R² Score', secondary_y=True, color='red')

# Update the layout with legend inside the plot and centered title
fig.update_layout(
    title=dict(
        text="Interactive Test RMSE and R² Score (Filtered Architectures)",
        x=0.5,  # Center the title
        xanchor='center',  # Anchor it to the center
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.05,
        xanchor='center',
        x=0.5,
        bgcolor='rgba(255,255,255,0.8)',  # Semi-transparent background
    ),
    hovermode='x unified',
)

# Display the figure
fig.show()